In [ ]:
# Import libraries

import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage import io
import time

# Data dir
ROOT_DIR = os.path.abspath("../")
DATASET_PATH = os.path.join(ROOT_DIR, "dataset")
EXPERIMENT_NAME = "exp-1"

if not os.path.exists(os.path.join(ROOT_DIR, "logs")):
    os.mkdir(os.path.join(ROOT_DIR, "logs"))

LOG_PATH = os.path.join(ROOT_DIR, "logs", EXPERIMENT_NAME)

if not os.path.exists(LOG_PATH):
    os.mkdir(LOG_PATH)
    
print(os.listdir(DATASET_PATH))

#### Read labels

In [ ]:
dt = pd.read_csv(os.path.join(DATASET_PATH, 'metadata.csv'))
dt = dt[["survival", "modality", "filename"]].dropna()
dt = dt[dt.modality != "CT"]

dt.head()

In [ ]:
# Labels
labels = dt[["survival"]].values

In [ ]:
len(labels)

In [ ]:
labels.shape

In [ ]:
image_path = dt["filename"].values
image_path[:10]

### Data loader

In [ ]:
# Classes for data loading and preprocessing
class COVIDChestXRayDataset:
    def __init__(
            self,
            datadir,
            csv_path,
            flag,
    ):
        
        # Patient names in folder
        #self.ids = sorted(os.listdir(datadir))
        # Sorted patient names in folder
        #self.images_fps = [os.path.join(datadir, image_id) for image_id in self.ids]
        

        # Read csv path
        csv = pd.read_csv(os.path.join(csv_path))
        csv = csv[["survival", "modality", "filename"]].dropna()
        csv = csv[csv.modality != "CT"]
        
        # Image names
        self.image_names = csv["filename"].values
        
        
        # Get labels
        self.labels = csv[["survival"]].values
        self.image_paths = [os.path.join(datadir, image_id) for image_id in self.image_names]
        
        # Split
        train_vols, test_vols, train_labels, test_labels = train_test_split(self.image_paths, self.labels, test_size=0.20, random_state=42)
        print(len(train_vols), len(test_vols))
        
        self.train_vols = train_vols
        self.test_vols = test_vols
        self.train_labels = train_labels
        self.test_labels = test_labels
        
        if flag == "train":
            self.image_paths = self.train_vols
            self.labels = self.train_labels
            self.ids = self.train_vols
            
        else:
            self.image_paths = self.test_vols
            self.labels = self.test_labels
            self.ids = self.test_vols
        
        
    def __getitem__(self, i):
        
        # Read data
        img = io.imread(self.image_paths[i])
        img = resize(img, (256, 256))
        
        img = img.astype(np.float32)
        img /= 255
        
        # Taken from https://github.com/mlmed/torchxrayvision/blob/master/torchxrayvision/datasets.py#L814
        # Check that images are 2D arrays
        if len(img.shape) > 2:
            img = img[:, :, 0]
        if len(img.shape) < 2:
            print("error, dimension lower than 2 for image")

        # Add color channel
        img = img[:, :, None]
        #img = np.dstack((img, img, img))
                   
        # Get labels
        gt = self.labels[i]
        if gt == "Y":
            gt = 1 # Survival: Yes
        else:
            gt = 0 # Survial: No
            
        gt = keras.utils.to_categorical(gt, 2)
        
        return img, gt
        
    def __len__(self):
        return len(self.ids)

In [ ]:
class Dataloder(keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        # Transpose list of lists
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        return batch
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)

In [ ]:
IMAGES_PATH = os.path.join(DATASET_PATH, "images/")
CSV_PATH = os.path.join(DATASET_PATH, "metadata.csv")

In [ ]:
# image path, csv path
train_dataset = COVIDChestXRayDataset(IMAGES_PATH, CSV_PATH, flag="train")
test_dataset = COVIDChestXRayDataset(IMAGES_PATH, CSV_PATH, flag="test")

In [ ]:
image, gt = train_dataset[0] 
image.shape, gt.shape

In [ ]:
gt

In [ ]:
image = np.squeeze(image)
plt.imshow(image, cmap='gray')

In [ ]:
train_dataloader = Dataloder(train_dataset, batch_size=1, shuffle=True)
test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=True)

In [ ]:
len(train_dataloader), len(test_dataloader)

In [ ]:
for batch_idx, (features, targets) in enumerate(train_dataloader):
    print(batch_idx, features.shape, targets.shape)

In [ ]:
for batch_idx, (features, targets) in enumerate(test_dataloader):
    print(batch_idx, features.shape, targets.shape)

In [ ]:
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPool2D, Activation,Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.models import load_model
from keras.optimizers import Adam

def vgg():
    base_model = VGG16(weights=None,include_top=False,pooling='avg',input_shape=(256, 256, 1))
    base_model.trainable = False
    X = base_model.output
    X.trainable = False
    predictions = Dense(2, activation='softmax')(X)
    
    #for layer in base_model.layers:
    #    layer.trainable=False
        
    model = Model(inputs=base_model.input, outputs=predictions)
        
    # Optimzer and loss
    optim = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    loss_func = 'binary_crossentropy'
    
    model.compile(optimizer=optim, loss=loss_func)
    return model



from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

def baseline():

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(256, 256, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['acc'])
    return model

model=None
model=vgg()
model.summary()

In [ ]:
# Define callbacks for learning rate scheduling, logging and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint('{}/{}.h5'.format(LOG_PATH, EXPERIMENT_NAME), monitor='val_loss', save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, verbose=1, patience=5, mode='min'), ## new_lr = lr * factor # 5
    keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, verbose=1, patience=15, mode='min', restore_best_weights=True), # 8
    keras.callbacks.CSVLogger('{}/training.csv'.format(LOG_PATH))
]

In [ ]:
start_time = time.time()


history = model.fit_generator(
    train_dataloader, 
    steps_per_epoch=len(train_dataloader), 
    epochs=50, 
    callbacks=callbacks, 
    validation_data=test_dataloader, 
    validation_steps=len(test_dataloader),  # val samples = batch size * no of steps
)

end_time = time.time()
print("--- Time taken to train : %s hours ---" % ((end_time - start_time)//3600))

In [ ]:
# Plot and save accuravy loss graphs together
def plot_loss(history):
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    
    plt.plot(epochs, loss, 'g')
    plt.plot(epochs, val_loss, 'y')
    plt.title('Loss')
    
    plt.ylabel('Rate')
    plt.xlabel('Epoch')
    
    plt.legend(['trainloss', 'valloss'], loc='lower right', fontsize=10)
    plt.grid(True)
    #plt.savefig('{}/{}_acc_loss_graph.jpg'.format(output_path, EXP_NAME), dpi=100)
    plt.show()


In [ ]:
plot_loss(model.history)

In [ ]:
y_pred = []
y_test = []

for batch_idx, (features, targets) in enumerate(test_dataloader):
    
    # Make predictions using trained model
    y_pred.append(model.predict(features)[0])
    y_test.append(targets[0])
    
y_pred = np.array(y_pred)
y_test = np.array(y_test)
print(y_pred.shape, y_test.shape)

In [ ]:
y_pred[0], y_test[0]

In [ ]:
# Convert ground truth to column values
y_test_flat = np.argmax(y_test, axis=1)
print("After flattening ground truth: ", y_test_flat.shape)

# Get labels from predictions
y_pred_flat = np.array([np.argmax(pred) for pred in y_pred]) # y_pred[1] -> probability for class 1 
print("Binarize probability values: ", y_pred_flat.shape)

In [ ]:
# Sanity check
print(y_test.shape, y_test_flat.shape, y_pred.shape, y_pred_flat.shape)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
# Accuracy

acc = accuracy_score(y_test_flat, y_pred_flat) * 100
print("Accuracy :", acc)

In [ ]:
# Average precision

from sklearn.metrics import average_precision_score
ap = average_precision_score(y_test, y_pred) * 100
print("Average precision :", ap)

In [ ]:
# Classification report

confusion_mtx = confusion_matrix(y_test_flat, y_pred_flat) 
print(confusion_mtx)
target_names = ['0', '1']
print(classification_report(y_test_flat, y_pred_flat, target_names=target_names))

In [ ]:
# Sensitivity and Specificity

cm = confusion_matrix(y_pred=y_pred_flat, y_true=y_test_flat)
total=sum(sum(cm))

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity*100 )

Specificity = cm[1,1]/(cm[1,1]+cm[0,1])
print('Specificity : ', Specificity*100 )

In [ ]:


y_pred[:10], y_test[:10]



In [ ]:
from sklearn.metrics import roc_auc_score
print('Area under ROC curve : ', roc_auc_score(y_test, y_pred) *100 )

In [ ]:
def deprocess_image(x):
    """Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    """
    x = x.copy()
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'th': #keras.backend.image_data_format()
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def normalize(x):
    """Utility function to normalize a tensor by its L2 norm"""
    return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)


def get_heatmap(gc):
  """Convert 2D heatmap to 3D for plotting"""
  # Get the color map
  cm = plt.get_cmap('jet')
  # Apply the colormap like a function to any array:
  gc3 = cm(gc)
  gc3 = gc3[:, :, :3].astype('float32') 
  return gc3

def grad_cam(input_model, image, cls, layer_name):
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.output[0, cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]
    # Normalize if necessary
    # grads = normalize(grads)
    gradient_function = K.function([input_model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # Process CAM
    cam = cv2.resize(cam, (256, 256), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

In [ ]:
from tqdm import tqdm


missclass_1 = []

for i in tqdm(range(len(y_test_flat))):
  # if predicted is 0 and actual is 1
  if y_pred_flat[i] == 0 and y_test_flat[i] == 1:
    missclass_1.append(x_test[i])

# take first 10 missclassified sampels
missclass_1 = np.array(missclass_1)

print("Number of missclassified samples:", missclass_1.shape)


missclass_1_gcam = []

#for ms in tqdm(missclass_1):
for batch_idx, (features, targets) in enumerate(test_dataloader):
    print(features.shape)
    features = 
    ms = np.expand_dims(np.squeeze(features), axis=0)
    # grad cam
    gc = grad_cam(model, ms, -1, 'block5_conv3')
    # convert to 3D
    gc = get_heatmap(gc)
    missclass_1_gcam.append(gc)

missclass_1_gcam = np.array(missclass_1_gcam)

print("Number of missclassified gradcam samples:", missclass_1_gcam.shape)


from numpy.random import rand
import matplotlib.pyplot as plt

results = np.concatenate((missclass_1, missclass_1_gcam), axis=0)

x = results
a, b = 1, 10
x = np.reshape(x, (a, b, 256, 256, 3))

test_data = x
r, c = test_data.shape[0], test_data.shape[1]
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')

plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
#plt.savefig("{}/{}_right_1gradcam.png".format(base_path, EXP_NAME), dpi=300)
plt.show()